# Introduction to trackintel

This tutorial will show you the most important functionalities of trackintel.

Trackintel focuses on the hierachical data model for movement data. The details of each level are explained in [readme.md](https://github.com/mie-lab/trackintel/blob/master/README.md)

![title](../docs/assets/hierarchy.png)


You can enter the hierarchy from any level and generate higher aggregation data and perform multiple analysis with your input data.

To get started, we import trackintel and other necessary libraries.



In [ ]:
# This is not needed if the trackintel library is installed. ==================
import sys
sys.path.append("..")
sys.path.append("../trackintel")
# =============================================================================

import trackintel as ti
import geopandas as gpd
import pandas as pd

from matplotlib import pyplot as plt
%matplotlib inline

# Data import
As a first step, we need to import the raw data and convert it into trackintel compatible format. In this example we use a stored GEOJSON file that contains raw positionfixes, and import it using GeoPandas _read_file()_ function. 

In [ ]:
# we read geojson files using geopandas function
gdf = gpd.read_file('./data/pfs_tutorial.geojson')
gdf.head(5)

gdf is currently a GeoDataFrame and not in the right shape to be used with trackintel.
If you try to create Positionfixes an error will occur.

In [ ]:
# an error will occur as the gdf is not yet transformed
# ti.Positionfixes(gdf)

From the error message it is clear that the column names are not yet recognized by trackintel. We provide helper functions to easily transform a GeoDataFrame into trackintel classes. One of them is [positionfixes_from_gpd](https://trackintel.readthedocs.io/en/latest/modules/io.html#geodataframe-import). The method allows to rename the columns and define a timezone. The index is assumed to be the unique identifier of the data.

In [ ]:
pfs = ti.io.read_positionfixes_gpd(gdf, tracked_at="time", user_id="User", geom_col="geometry", tz='UTC')
# pfs is of class Positionfixes that has the generate_staypoints method
pfs.generate_staypoints()

This is the recommended way to import data. We also provide functions to directly load data from csv files: [read_positionfixes_csv](https://trackintel.readthedocs.io/en/latest/modules/io.html#csv-file-import). Or load data from Geolife dataset: [read_geolife](https://trackintel.readthedocs.io/en/latest/modules/io.html#trackintel.io.dataset_reader.read_geolife)

In [ ]:
# load from csv
ti.io.read_positionfixes_csv('./data/pfs.csv', tz='UTC', sep=";", index_col="id", crs="EPSG:4326")

In [ ]:
# Load example geolife dataset
pfs, _ = ti.io.read_geolife('../tests/data/geolife_long')
# plot the raw positionfixes
ti.plot(positionfixes=pfs, plot_osm=True)

Once imported trackintel classes like Positionfixes can be used like GeoDataFrames

In [ ]:
pfs["new_column"] = 1
pfs.head()

We provide import data functions for each data level. Detailed information can be found in the [I/O module](https://trackintel.readthedocs.io/en/latest/modules/io.html#)

# Data model generation
All trackintel classes have of course different methods available depending on the data level.
Relevant information can be found in the [Model documentation](https://trackintel.readthedocs.io/en/latest/modules/model.html). Equivalent functions exist for every method and can be called directly from the respective module.

As an example, we generate the staypoints and triplegs with the raw positionfix data.

In [ ]:
# generate staypoints from positionfixes. The two lines are equivalent
pfs, sp = pfs.generate_staypoints(method='sliding', dist_threshold=100, time_threshold=5.0, include_last=False)
# pfs, sp = ti.preprocessing.generate_staypoints(pfs, method='sliding', dist_threshold=100, time_threshold=5.0, include_last=False)

# generate triplegs from positionfixes and staypoints. 
pfs, tpls = pfs.generate_triplegs(sp, method='between_staypoints', gap_threshold=15)
# pfs, tpls = ti.preprocessing.generate_triplegs(pfs, sp, method="between_staypoints", gap_threshold=15)

# plot the triplegs together with staypoints and positionfixes
ti.plot(triplegs=tpls, positionfixes=pfs, staypoints=sp, radius_sp=30, plot_osm=True)

We can also generate trip as aggregation of triplegs to a single activity, see [generate_trips](https://trackintel.readthedocs.io/en/latest/modules/preprocessing.html#trackintel.preprocessing.triplegs.generate_trips) for more information.

In [ ]:
# infer activity label based on duration, here any staypoint larger than 15 min is considered as an activity
sp = sp.create_activity_flag(time_threshold=15)

# generate trips with triplegs and staypoints with activity labels
sp, tpls, trips = ti.preprocessing.generate_trips(sp, tpls, gap_threshold=15)

trips.head(5)

# Analysis
The [analysis module](https://trackintel.readthedocs.io/en/latest/modules/analysis.html) in *trackintel* focuses on the semantic enrichment and quantitative analysis of human movement data. As an example, we provide labeling functions to infer semantic labels of staypoints and triplegs.


In [ ]:
# infer activity label based on duration
sp = sp.create_activity_flag(time_threshold=15)

# the result is the original staypoint with a column 'is_activity', indicating whether the stp correspond to an activity
sp.head(5)

In [ ]:
# infer transport mode based on speed 
tpls = tpls.predict_transport_mode()

# the result is the original tripleg with a column 'mode'
tpls.head(5)

The tracking coverage of each user can be obtained using [temporal_tracking_quality()](https://trackintel.readthedocs.io/en/latest/modules/analysis.html#trackintel.analysis.tracking_quality.temporal_tracking_quality)

In [ ]:
# we concate sp and tpls to get the whole mobility trace
trace = pd.concat([sp, tpls])

# calculate the overall tracking coverage
ti.analysis.temporal_tracking_quality(trace, granularity='all')

In [ ]:
# or coverage by day or hour
ti.analysis.temporal_tracking_quality(trace, granularity='hour')
ti.analysis.temporal_tracking_quality(trace, granularity='day')

We can also calculate the modal split of triplegs using [calculate_modal_split()](https://trackintel.readthedocs.io/en/latest/modules/analysis.html#trackintel.analysis.modal_split.calculate_modal_split) and visualize the results using [plot_modal_split()](https://trackintel.readthedocs.io/en/latest/modules/visualization.html#trackintel.visualization.modal_split.plot_modal_split)

In [ ]:
# for the modal split we load a special subset of geolife dataset with tripleg modes
pfs, labels = ti.io.read_geolife('../tests/data/geolife_modes')

# sp and tpls are generated 
pfs, sp = pfs.generate_staypoints(method="sliding", dist_threshold=25, time_threshold=5)
_, tpls = pfs.generate_triplegs(sp, method="between_staypoints")

# the mode labels are assigned to each tpls
tpls_with_modes = ti.io.geolife_add_modes_to_triplegs(tpls, labels)
tpls_with_modes.head(5)

In [ ]:
# the modal split can then be calculated
modal_split = ti.analysis.calculate_modal_split(tpls_with_modes, metric="count", freq="D", per_user=False, norm=True)

# and visualized
ti.plot_modal_split(modal_split, date_fmt_x_axis='%d', y_label='Percentage of daily count', x_label='days')
plt.show()

# Save results
The generated data can be exported as csv files using [`to_csv()` functions](https://trackintel.readthedocs.io/en/latest/modules/io.html#csv-file-export) registered to each data model. For each function in the visualization module, we provide an `filename` parameter to directly save the figures as a file.

In [ ]:
# save datamodels to csv files
pfs.to_csv('./out/pfs_tutorial.csv')
sp.to_csv('./out/sp_tutorial.csv')
tpls.to_csv('./out/tpls_tutorial.csv')
trips.to_csv('./out/trips_tutorial.csv')

# filename will save the generated plot as a file
ti.plot(filename='./out/pfs.png', positionfixes=pfs)

For database users, we provide functions and instructions to support the import and export from PostGIS databases. Further information can be found in the [trackintel documentation](https://trackintel.readthedocs.io/en/latest/). 

Thanks for using trackintel. If you discover any bug and/or have an idea for a usful funcionality feel free to open an [issue](https://github.com/mie-lab/trackintel/issues)